In [9]:
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
import pandas as pd
import os
import json

In [5]:
model_path = "../bioSent2Vec_model/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"

model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [6]:
stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

In [27]:
def vectorize(input_path):
    with open(input_path) as f:
        user_text_list = f.readlines()

    user_vect_text_dict = {}
    for u_txt in user_text_list:
        id, text = u_txt.split("_")
        text_list = text.split(",")
        vect = []
        for text in text_list:
            sentence = preprocess_sentence(text)
            sentence_vector = model.embed_sentence(sentence)
            vect.append(sentence_vector.tolist()[0])
        user_vect_text_dict[id] = vect
    return user_vect_text_dict
    
def write_json(dict, output_path):
    with open(output_path, "w") as outfile:
        json.dump(dict, outfile)

In [31]:
fold0_dict = vectorize('../diseaseDiagnosis/Dataset/Fold0/TrainingSet.txt')
write_json(fold0_dict, "user_vect_fold0.json")

In [33]:
fold0_dict = vectorize('input.txt')
write_json(fold0_dict, "user_vect_input.json")